Import Statements

In [149]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, avg, count, sum, datediff, round, col, coalesce, current_date, trim, lower, year, month, concat, lit, substring, length, lpad, udf
from pyspark.sql.types import StringType
from google.colab import drive

Creating PySpark Session

In [150]:
spark = SparkSession.builder \
                    .appName("Assessment2") \
                    .getOrCreate()
spark

Mounting Google Drive

In [151]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Tasks

### 1. Ingestion & Exploration

Read all 3 files (CSV + JSON) using PySpark.

In [152]:
# Reading employee data
employee_df = spark.read.csv('/content/drive/MyDrive/PysparkAssessment/Assessment_2/employees.csv', header=True, inferSchema=True)

# Reading order data
attendance_df =  spark.read.csv('/content/drive/MyDrive/PysparkAssessment/Assessment_2/attendance.csv', header=True, inferSchema=True)

# Reading bonus data
bonus_df = spark.read.option("multiline", "true").json('/content/drive/MyDrive/PysparkAssessment/Assessment_2/bonuses.json')


Displaying Schemas

In [153]:
print("\n Employee Data Schema")
employee_df.printSchema()

print("\n Attendance Data Schema")
attendance_df.printSchema()

print("\n Bonous Data Schema")
bonus_df.printSchema()


 Employee Data Schema
root
 |-- EmpID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- JoinDate: date (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- ManagerID : string (nullable = true)


 Attendance Data Schema
root
 |--  EmpID: integer (nullable = true)
 |-- Date: date (nullable = true)
 |-- Status : string (nullable = true)


 Bonous Data Schema
root
 |-- Bonus: long (nullable = true)
 |-- EmpID: long (nullable = true)
 |-- Year: long (nullable = true)



Displaying sample data

In [154]:
print("\n Employee Sample Data")
employee_df.show(5)

print("\n Attendance Sample Data")
attendance_df.show(5)

print("\n Bonus Sample Data")
bonus_df.show(5)


 Employee Sample Data
+-----+------+-----------+----------+------+----------+
|EmpID|  Name| Department|  JoinDate|Salary|ManagerID |
+-----+------+-----------+----------+------+----------+
|    1| Anita|         HR|2021-05-01| 55000|          |
|    2|   Raj|Engineering|2020-03-15| 80000|        1 |
|    3|Simran|Engineering|2022-07-10| 75000|        1 |
|    4| Aamir|  Marketing|2019-11-20| 60000|        1 |
|    5| Nisha|         HR|2023-01-05| 50000|         1|
+-----+------+-----------+----------+------+----------+


 Attendance Sample Data
+------+----------+--------+
| EmpID|      Date| Status |
+------+----------+--------+
|     1|2024-04-01|Present |
|     1|2024-04-02|Present |
|     2|2024-04-01| Absent |
|     2|2024-04-02|Present |
|     3|2024-04-01|Present |
+------+----------+--------+
only showing top 5 rows


 Bonus Sample Data
+-----+-----+----+
|Bonus|EmpID|Year|
+-----+-----+----+
| 5000|    1|2023|
| 7000|    2|2023|
| 6500|    3|2023|
| 6000|    4|2023|
| 4000| 

 Count distinct departments.

In [155]:
employee_df.select("department").distinct().count()

3

### 2.  DataFrame Operations

Add a column TenureYears using datediff() and round().

In [156]:
employee_df = employee_df.withColumn("TenureYears",
                                      round(datediff(current_date(), col("JoinDate")) / 365, 1)
                                    )
employee_df.select("EMPID", "Name", "Department", "JoinDate", "TenureYears").show()

+-----+------+-----------+----------+-----------+
|EMPID|  Name| Department|  JoinDate|TenureYears|
+-----+------+-----------+----------+-----------+
|    1| Anita|         HR|2021-05-01|        4.1|
|    2|   Raj|Engineering|2020-03-15|        5.2|
|    3|Simran|Engineering|2022-07-10|        2.9|
|    4| Aamir|  Marketing|2019-11-20|        5.6|
|    5| Nisha|         HR|2023-01-05|        2.4|
+-----+------+-----------+----------+-----------+



 Calculate TotalCompensation = Salary + Bonus.

In [157]:
emp_bonus_df = employee_df.join(bonus_df.select("EmpID", "Bonus"), on="EmpID", how="left")

emp_bonus_df = emp_bonus_df.withColumn("TotalCompensation", emp_bonus_df.Salary + emp_bonus_df.Bonus)

emp_bonus_df.select("EmpID", "Name", "Salary", "Bonus", "TotalCompensation").show()

+-----+------+------+-----+-----------------+
|EmpID|  Name|Salary|Bonus|TotalCompensation|
+-----+------+------+-----+-----------------+
|    1| Anita| 55000| 5000|            60000|
|    2|   Raj| 80000| 7000|            87000|
|    3|Simran| 75000| 6500|            81500|
|    4| Aamir| 60000| 6000|            66000|
|    5| Nisha| 50000| 4000|            54000|
+-----+------+------+-----+-----------------+



 Filter employees with more than 2 years in the company.

In [158]:
employee_df.filter(col("TenureYears") > 2) \
           .select("EmpID", "Name", "Department", "JoinDate", "TenureYears") \
           .show()

+-----+------+-----------+----------+-----------+
|EmpID|  Name| Department|  JoinDate|TenureYears|
+-----+------+-----------+----------+-----------+
|    1| Anita|         HR|2021-05-01|        4.1|
|    2|   Raj|Engineering|2020-03-15|        5.2|
|    3|Simran|Engineering|2022-07-10|        2.9|
|    4| Aamir|  Marketing|2019-11-20|        5.6|
|    5| Nisha|         HR|2023-01-05|        2.4|
+-----+------+-----------+----------+-----------+



Show employees who report to a manager (ManagerID is not null).

In [185]:
employee_df.filter((col("ManagerID").isNotNull()) & (col("ManagerID") != "")) \
           .select("EmpID", "Name", "ManagerID") \
           .show()

+-----+------+----------+
|EmpID|  Name| ManagerID|
+-----+------+----------+
|    1| Anita|No Manager|
|    2|   Raj|         1|
|    3|Simran|         1|
|    4| Aamir|         1|
|    5| Nisha|         1|
+-----+------+----------+



### 3. Aggregation

 Average salary per department

In [160]:
employee_df.groupBy("Department") \
           .agg(avg("Salary").alias("AverageSalary")) \
           .orderBy("AverageSalary", ascending = False) \
           .show()

+-----------+-------------+
| Department|AverageSalary|
+-----------+-------------+
|Engineering|      77500.0|
|  Marketing|      60000.0|
|         HR|      52500.0|
+-----------+-------------+



Number of employees under each manager.

In [187]:
employee_df = employee_df.withColumn("ManagerID", trim(col("ManagerID")))
employee_df.groupBy("ManagerID") \
           .agg(count("EmpID").alias("Total_employees")) \
           .filter((col("ManagerID").isNotNull()) & (col("ManagerID") != "")) \
           .orderBy("Total_employees", ascending=False) \
           .show()

+----------+---------------+
| ManagerID|Total_employees|
+----------+---------------+
|         1|              4|
|No Manager|              1|
+----------+---------------+



 Count of absences per employee.

In [165]:
attendance_df = attendance_df.withColumn("Status", trim(col("Status"))) \
                             .withColumnRenamed("Status ", "Status") \
                             .withColumnRenamed(" EmpID", "EmpID")


attendance_df.filter(col("Status") == "Absent") \
             .groupBy("EmpID") \
             .agg(count("*").alias("AbsenceCount")) \
             .orderBy("AbsenceCount", ascending=False) \
             .show()

+-----+------------+
|EmpID|AbsenceCount|
+-----+------------+
|    4|           2|
|    2|           1|
+-----+------------+



### 4. Joins

Join employees and attendance → Get attendance % (Present days / Total days).

In [184]:
employee_attendance_df = employee_df.join(attendance_df, on="EmpID", how="left")

employee_attendance_df = employee_attendance_df.groupBy("EmpID", "Name", "Department") \
                                                .agg(
                                                    count("Status").alias("TotalDays"),
                                                    count(when(col("Status") == "Present", True)).alias("PresentDays")
                                                ) \
                                                .withColumn("AttendancePercentage", round((col("PresentDays") / col("TotalDays")) * 100, 2))

employee_attendance_df.show()

+-----+------+-----------+---------+-----------+--------------------+
|EmpID|  Name| Department|TotalDays|PresentDays|AttendancePercentage|
+-----+------+-----------+---------+-----------+--------------------+
|    2|   Raj|Engineering|        2|          1|                50.0|
|    5| Nisha|         HR|        2|          2|               100.0|
|    1| Anita|         HR|        2|          2|               100.0|
|    4| Aamir|  Marketing|        2|          0|                 0.0|
|    3|Simran|Engineering|        2|          2|               100.0|
+-----+------+-----------+---------+-----------+--------------------+



Join employees and bonuses → Show top 3 employees by TotalCompensation.

In [167]:
employee_bonus_df = employee_df.join(bonus_df, on="EmpID", how="left")
employee_bonus_df = employee_bonus_df.withColumn("TotalCompensation", col("Salary") + col("Bonus"))

employee_bonus_df.select("EmpID", "Name", "Department", "Salary", "Bonus", "TotalCompensation").show()


+-----+------+-----------+------+-----+-----------------+
|EmpID|  Name| Department|Salary|Bonus|TotalCompensation|
+-----+------+-----------+------+-----+-----------------+
|    1| Anita|         HR| 55000| 5000|            60000|
|    2|   Raj|Engineering| 80000| 7000|            87000|
|    3|Simran|Engineering| 75000| 6500|            81500|
|    4| Aamir|  Marketing| 60000| 6000|            66000|
|    5| Nisha|         HR| 50000| 4000|            54000|
+-----+------+-----------+------+-----+-----------------+



 Multi-level join: employees + bonuses + attendance

In [168]:
multi_df = emp_bonus_df.join(attendance_df, on="EmpID", how="left")
multi_df.show()

+-----+------+-----------+----------+------+----------+-----------+-----+-----------------+----------+-------+
|EmpID|  Name| Department|  JoinDate|Salary|ManagerID |TenureYears|Bonus|TotalCompensation|      Date| Status|
+-----+------+-----------+----------+------+----------+-----------+-----+-----------------+----------+-------+
|    1| Anita|         HR|2021-05-01| 55000|          |        4.1| 5000|            60000|2024-04-02|Present|
|    1| Anita|         HR|2021-05-01| 55000|          |        4.1| 5000|            60000|2024-04-01|Present|
|    2|   Raj|Engineering|2020-03-15| 80000|        1 |        5.2| 7000|            87000|2024-04-02|Present|
|    2|   Raj|Engineering|2020-03-15| 80000|        1 |        5.2| 7000|            87000|2024-04-01| Absent|
|    3|Simran|Engineering|2022-07-10| 75000|        1 |        2.9| 6500|            81500|2024-04-02|Present|
|    3|Simran|Engineering|2022-07-10| 75000|        1 |        2.9| 6500|            81500|2024-04-01|Present|
|

### 5. String & Date Functions

 Extract year and month from JoinDate

In [169]:
employee_df = employee_df.withColumn("JoinYear", year(col("JoinDate"))) \
                         .withColumn("JoinMonth", month(col("JoinDate")))

employee_df.select("EmpID", "Name", "Department","JoinDate", "JoinYear", "JoinMonth").show()

+-----+------+-----------+----------+--------+---------+
|EmpID|  Name| Department|  JoinDate|JoinYear|JoinMonth|
+-----+------+-----------+----------+--------+---------+
|    1| Anita|         HR|2021-05-01|    2021|        5|
|    2|   Raj|Engineering|2020-03-15|    2020|        3|
|    3|Simran|Engineering|2022-07-10|    2022|        7|
|    4| Aamir|  Marketing|2019-11-20|    2019|       11|
|    5| Nisha|         HR|2023-01-05|    2023|        1|
+-----+------+-----------+----------+--------+---------+



Mask employee names using regex.

In [170]:
masked_df = employee_df.withColumn("MaskedName",
                                    concat(substring(col("Name"), 1, 3), lit("***"))
                                  )

masked_df.select("Name", "MaskedName").show()

+------+----------+
|  Name|MaskedName|
+------+----------+
| Anita|    Ani***|
|   Raj|    Raj***|
|Simran|    Sim***|
| Aamir|    Aam***|
| Nisha|    Nis***|
+------+----------+



 Use substring() to create EmpCode like "EMP001"

In [171]:
employee_df = employee_df.withColumn("EmpCode",
                                     concat(lit("EMP"),lpad(col("EmpID").cast("string"), 3, "0"))
                                    )

employee_df.select("EmpID", "EmpCode", "Name", "Department").show()


+-----+-------+------+-----------+
|EmpID|EmpCode|  Name| Department|
+-----+-------+------+-----------+
|    1| EMP001| Anita|         HR|
|    2| EMP002|   Raj|Engineering|
|    3| EMP003|Simran|Engineering|
|    4| EMP004| Aamir|  Marketing|
|    5| EMP005| Nisha|         HR|
+-----+-------+------+-----------+



### 6. Conditional & Null Handling

Use when/otherwise to label performance: \
 “High” if Bonus > 6000 \
 “Medium” if 4000–6000 \
 “Low” otherwise

In [172]:
bonus_df = bonus_df.withColumn("Performance",
                                 when(col("Bonus") > 6000, "High")
                                .when((col("Bonus") >= 4000) & (col("Bonus") <= 6000), "Medium")
                                .otherwise("Low")
                              )
bonus_df.show()

+-----+-----+----+-----------+
|Bonus|EmpID|Year|Performance|
+-----+-----+----+-----------+
| 5000|    1|2023|     Medium|
| 7000|    2|2023|       High|
| 6500|    3|2023|       High|
| 6000|    4|2023|     Medium|
| 4000|    5|2023|     Medium|
+-----+-----+----+-----------+



Handle missing ManagerID using fillna("No Manager")

In [174]:
employee_df = employee_df.withColumn("ManagerID",
                                      when(col("ManagerID") == "", "No Manager")
                                      .otherwise(col("ManagerID")))

employee_df.select("EmpID", "Name", "Department","ManagerID").show()

+-----+------+-----------+----------+
|EmpID|  Name| Department| ManagerID|
+-----+------+-----------+----------+
|    1| Anita|         HR|No Manager|
|    2|   Raj|Engineering|         1|
|    3|Simran|Engineering|         1|
|    4| Aamir|  Marketing|         1|
|    5| Nisha|         HR|         1|
+-----+------+-----------+----------+



### 7. Spark SQL

Create and use database hr.

In [175]:
spark.sql("""
          CREATE DATABASE  IF NOT EXISTS hr
""")

spark.sql("USE hr")

DataFrame[]

Save all DataFrames as tables: employees, attendance, bonuses.

In [176]:
employee_df.write.mode("overwrite").saveAsTable("employees")
attendance_df.write.mode("overwrite").saveAsTable("attendance")
bonus_df.write.mode("overwrite").saveAsTable("bonuses")

 Top paid employee in each department.

In [177]:
spark.sql("""
          SELECT
            EmpID,
            Name,
            Department,
            Salary
            FROM (
                  SELECT
                    EmpID,
                    Name,
                    Department,
                    Salary,
                    ROW_NUMBER() OVER (PARTITION BY Department ORDER BY Salary DESC) AS row_number
                  FROM employees
                ) t
            WHERE row_number = 1
""").show()

+-----+-----+-----------+------+
|EmpID| Name| Department|Salary|
+-----+-----+-----------+------+
|    2|  Raj|Engineering| 80000|
|    1|Anita|         HR| 55000|
|    4|Aamir|  Marketing| 60000|
+-----+-----+-----------+------+



Attendance rate by department.

In [178]:
spark.sql("""
            SELECT
              e.Department,
              ROUND(SUM(CASE WHEN a.Status = 'Present' THEN 1 ELSE 0 END) * 100.0 / COUNT(a.Status), 2) AS Attendance_percentage
            FROM employees e
            JOIN attendance a ON e.EmpID = a.EmpID
            GROUP BY e.Department
         """).show()


+-----------+---------------------+
| Department|Attendance_percentage|
+-----------+---------------------+
|Engineering|                75.00|
|         HR|               100.00|
|  Marketing|                 0.00|
+-----------+---------------------+



 Employees joined after 2021 with salary > 70,000.

In [180]:
spark.sql("""
          SELECT
            EmpID,
            Name,
            Department,
            JoinDate
          FROM employees
          WHERE JoinDate > '2021-12-31' AND Salary > 70000
""").show()

+-----+------+-----------+----------+
|EmpID|  Name| Department|  JoinDate|
+-----+------+-----------+----------+
|    3|Simran|Engineering|2022-07-10|
+-----+------+-----------+----------+



### 8. Advanced

Use a UDF to classify department as "Tech" vs "Non-Tech".

In [132]:
def classify_departments(department):
    tech_depts = ["Engineering", "IT", "Development", "Tech Support"]

    if department in tech_depts:
        return "Tech"
    return "Non-Tech"


classify_department_udf = udf(classify_departments, StringType())

employee_df = employee_df.withColumn("DeptCategory", classify_department_udf(employee_df.Department))

employee_df.select("EmpID", "Name", "Department", "DeptCategory").show()

+-----+------+-----------+------------+
|EmpID|  Name| Department|DeptCategory|
+-----+------+-----------+------------+
|    1| Anita|         HR|    Non-Tech|
|    2|   Raj|Engineering|        Tech|
|    3|Simran|Engineering|        Tech|
|    4| Aamir|  Marketing|    Non-Tech|
|    5| Nisha|         HR|    Non-Tech|
+-----+------+-----------+------------+



 Create a view emp_attendance_summary.

In [181]:
spark.sql("""
    CREATE OR REPLACE VIEW emp_attendance_summary AS
    SELECT

      e.EmpID,
      e.Name,
      e.Department,
      COUNT(a.Status) AS Total_days,
      SUM(CASE WHEN a.Status = 'Present' THEN 1 ELSE 0 END) AS Present_days,
      ROUND(SUM(CASE WHEN a.Status = 'Present' THEN 1 ELSE 0 END) * 100.0 / COUNT(a.Status), 2) AS Attendance_percentage

    FROM employees e
    LEFT JOIN attendance a ON e.EmpID = a.EmpID
    GROUP BY e.EmpID, e.Name, e.Department
""")
spark.sql("SELECT * FROM emp_attendance_summary").show()

+-----+------+-----------+----------+------------+---------------------+
|EmpID|  Name| Department|Total_days|Present_days|Attendance_percentage|
+-----+------+-----------+----------+------------+---------------------+
|    2|   Raj|Engineering|         2|           1|                50.00|
|    5| Nisha|         HR|         2|           2|               100.00|
|    1| Anita|         HR|         2|           2|               100.00|
|    4| Aamir|  Marketing|         2|           0|                 0.00|
|    3|Simran|Engineering|         2|           2|               100.00|
+-----+------+-----------+----------+------------+---------------------+



Save it as Parquet partitioned by Department.

In [183]:
attendance_summary_df = spark.table("emp_attendance_summary")
attendance_summary_df.write.mode("overwrite").partitionBy("Department").parquet("/mnt/data/attendance_summary_parquet")

read_parquet_df = spark.read.parquet("/mnt/data/attendance_summary_parquet")
read_parquet_df.show()


+-----+------+----------+------------+---------------------+-----------+
|EmpID|  Name|Total_days|Present_days|Attendance_percentage| Department|
+-----+------+----------+------------+---------------------+-----------+
|    5| Nisha|         2|           2|               100.00|         HR|
|    1| Anita|         2|           2|               100.00|         HR|
|    2|   Raj|         2|           1|                50.00|Engineering|
|    3|Simran|         2|           2|               100.00|Engineering|
|    4| Aamir|         2|           0|                 0.00|  Marketing|
+-----+------+----------+------------+---------------------+-----------+

